In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT


In [15]:
def run(env, steps=100):
    done = True
    for step in range(steps):
        if done:
            state = env.reset()
        state, reward, done, info = env.step(env.action_space.sample())
        env.render()

        
def runAndClose(env, steps=100):
    run(env, steps)
    env.close()
    

In [16]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
runAndClose(env)


In [6]:
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

In [19]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, [["right"], ["right", "A"]])


In [20]:
env.reset()
next_state, reward, done, info = env.step(action=0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")
runAndClose(env)

(240, 256, 3),
 0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}


In [27]:
class NullWrapper(gym.Wrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         self.env = env # don't need it as env is already available in self.env
    pass

    def step(self, action):
        next_state, reward, done, info = self.env.step(action)
        # modify ...
        return next_state, reward, done, info

In [28]:
env = gym_super_mario_bros.make('SuperMarioBros2-v0')
env = JoypadSpace(env, [["right"], ["right", "A"]])
env = NullWrapper(env)
runAndClose(env)

Exception: wtf